In [1]:
import pandas as pd #1
import numpy as np
import glob,re, os
import shutil

username = 'john.tan'
df_invoice = pd.read_excel(r"C:/Users/"+username+r"/Downloads/esker_merged/invoice_rename.xlsx", sheet_name='invoice_rename', engine='openpyxl')
directory_invoice = r"C:/Users/"+username+"/Downloads/esker_merged/" # Directory containing the PDF files. 'Documents'

list_invoice_numbr = df_invoice['invoice'].tolist()
list_invoice_name = df_invoice['invoice_name'].tolist()
list_company_code = df_invoice['company_code'].tolist()
list_invoice_date = df_invoice['invoice_date'].tolist()

list_invoice_numbr = [str(i) for i in list_invoice_numbr]
list_invoice_numbr = [str(i).replace('/', '_') for i in list_invoice_numbr]

list_of_lists = [list_invoice_numbr, list_invoice_name]
list_of_lists_2 = [list_invoice_numbr, list_company_code]
list_of_lists_3 = [list_invoice_numbr, list_invoice_date]

def get_dict_invoice_name_number(list_of_lists):
        return {str(z[0]): (z[1]) for z in zip(*list_of_lists)}

def get_dict_invoice_name_company_code(list_of_lists_2):
        return {str(z[0]): (z[1]) for z in zip(*list_of_lists_2)}

def get_dict_invoice_name_invoice_date(list_of_lists_3):
        return {str(z[0]): (z[1]) for z in zip(*list_of_lists_3)}

dict_invoice_name_number = get_dict_invoice_name_number(list_of_lists)
dict_invoice_name_company_code = get_dict_invoice_name_company_code(list_of_lists_2)
dict_invoice_name_invoice_date = get_dict_invoice_name_invoice_date(list_of_lists_3)


def format_dictionary_keys_in_place(dictionary_invoice):
        # Iterate over a copy of the keys to avoid RuntimeError
        for key in list(dictionary_invoice.keys()):
            new_key = key.replace(':', '_').replace('/', '_')
            if new_key != key:  # Only update if the key has changed
                dictionary_invoice[new_key] = dictionary_invoice.pop(key)
        return dictionary_invoice  # Return the modified dictionary

format_dict_invoice_name_number = format_dictionary_keys_in_place(dict_invoice_name_number)
format_dict_invoice_name_company_code = format_dictionary_keys_in_place(dict_invoice_name_company_code)
format_dict_invoice_name_invoice_date = format_dictionary_keys_in_place(dict_invoice_name_invoice_date)


from pandas import Timestamp # Assuming Timestamp is from pandas #2

format_dict_invoice_name_company_code = {k: v for k, v in format_dict_invoice_name_company_code.items() if k in format_dict_invoice_name_number}

format_dict_invoice_name_invoice_date = {k: v for k, v in format_dict_invoice_name_invoice_date.items() if k in format_dict_invoice_name_number}

def format_invoice_dates(dict_invoice_name_invoice_date):
    """
    Converts Timestamp values in a dictionary to string dates formatted as "%Y%m%d".
    Args:
      dict_invoice_numbr_invoice_date: A dictionary where keys are invoice numbers
                                      and values are Timestamp objects.
    Returns:
      A new dictionary with the same keys and formatted string dates as values.
    """
    formatted_dict = {}
    for invoice_name, invoice_date in dict_invoice_name_invoice_date.items():
        # Ensure the value is a Timestamp object before formatting
        if isinstance(invoice_date, (Timestamp)): #datetime
            formatted_dict[invoice_name] = invoice_date.strftime("%Y%m%d")
        else:
            # Handle cases where the value might not be a Timestamp as expected
            # You could raise an error, skip, or try a different conversion
            formatted_dict[invoice_name] = str(invoice_date) # Or None, or skip
    return formatted_dict

# Convert the dictionary
formatted_dates_dict = format_invoice_dates(dict_invoice_name_invoice_date)
#print(formatted_dates_dict) # Print the result


def rename_pdf_files(directory_invoice, list_invoice_numbr, dict_invoice_name_number, dict_invoice_name_company_code, formatted_dates_dict):
    pdf_rename =0
    for filename in os.listdir(directory_invoice):
            # Check if the file is a PDF and its name (without extension) is in list_invoice_name
            if (filename.endswith(".pdf")): 
                # Get the invoice num (filename without extension)
                invoice_numbr = os.path.splitext(filename)[0]
                #print(invoice_numbr)
                # Check if the filename contains '_merged'
                if '_merged' in invoice_numbr:
                    # Remove '_merged' from the filename
                    invoice_numbr = invoice_numbr.replace('_merged', '')
                    if invoice_numbr in list_invoice_numbr:                        
                
                        # Get the corresponding value from dict_invoice_name_number
                        value = dict_invoice_name_number.get(str(invoice_numbr), "")  
                        invoice_date = formatted_dates_dict.get(str(invoice_numbr), "")
                        company_code = dict_invoice_name_company_code.get(str(invoice_numbr), "")                
            
                        new_filename = f"{value}_{invoice_numbr}_{invoice_date}_{company_code}.pdf" # Construct the new filename
                        # Get the full paths for the old and new filenames
                        old_file_path = os.path.join(directory_invoice, filename)
                        new_file_path = os.path.join(directory_invoice, new_filename)

                        try:
                            os.rename(old_file_path, new_file_path) # Rename the file         
                            #print(f"Renamed: {filename} -> {new_filename}")
                            pdf_rename +=1
                        except Exception as e:
                            print(f"failed to rename {filename}: {e}")
                            continue
    print(f'done rename {pdf_rename} pdf files')
    return pdf_rename        


from datetime import datetime, timedelta

def create_folder_today(downloads_drive):
    # Get today date with format 'yyyymmdd'
    today = datetime.today().strftime('%Y%m%d')
    today_str = today
    # Remove leading zero if day is single digit (not needed for yyyymm%d format, but kept for consistency)
    today_str = today_str[:-2] + str(int(today_str[-2:]))

    # Create folder if not exists
    folder_path = os.path.join(downloads_drive, today_str)
    os.makedirs(folder_path, exist_ok=True)

    # Move all '.pdf' files to this folder
    for filename in os.listdir(downloads_drive):
        if filename.endswith(".pdf"):
            file_path = os.path.join(downloads_drive, filename)
            try:
                shutil.move(file_path, folder_path)
                #print(f"File {filename} moved successfully to {folder_path}")
            except Exception as e:
                print(f"Error moving file {filename}: {str(e)}")


def move_folders(downloads_drive, specified_drive):
    # Get yesterday and today date with format 'yyyymmdd'
    today = datetime.today().strftime('%Y%m%d')
    yesterday = (datetime.today() - timedelta(days=1)).strftime('%Y%m%d')

    # Remove leading zero if day is single digit (not needed for yyyymmdd format, but kept for consistency)
    today_str = today
    yesterday_str = yesterday
    list_dates=[today_str,yesterday_str]
    list_dates = [date[:-2] + str(int(date[-2:])) for date in list_dates]    

    # Check 'downloads_drive' if folder 'yyyymmdd' exist for yesterday and today
    for date_str in list_dates:
        folder_path = os.path.join(downloads_drive, date_str)
        if os.path.exists(folder_path) and os.path.isdir(folder_path):
            # Move entire folder (with all files) to 'specified_drive'
            destination_path = os.path.join(specified_drive, date_str)
            # Ensure the destination directory exists
            os.makedirs(os.path.dirname(destination_path), exist_ok=True)
            try:
                shutil.move(folder_path, destination_path)
                #print(f"Folder {date_str} moved successfully to {specified_drive}")
            except Exception as e:
                print(f"Error moving folder {date_str}: {str(e)}")

# Example usage
downloads_drive = r'C:/Users/'+username+'/Downloads/esker_merged/'
#share_point_drive = r"C:/Users/john.tan/Cogent Holdings Pte. Ltd/Cogent Group - 002 Cogent - Invoice/SG77 - SHCL/GIRO - CDAS/"
share_point_drive = r"C:/Users/"+username+"/Documents/AP/"

if __name__ == "__main__":
    rename_pdf_files(directory_invoice, list_invoice_numbr, dict_invoice_name_number, dict_invoice_name_company_code, formatted_dates_dict)

    create_folder_today(downloads_drive)
    move_folders(downloads_drive, share_point_drive)

#run

done rename 147 pdf files
